# Robust MDP

---

今回はこの[論文](https://arxiv.org/abs/2112.01506)をコードも含めて解説してきます．

Robust MDPは従来のMDPと異なる点は，遷移確率行列Pに不確実性を持たせ，現実世界のモデルに適用した時に，シュミレータとのギャップを減らすのがモチベーションでできたMDPです．

---

### **予備知識**

robust MDP は次のMDPを考えます．

**$M = (S,A,r,P,\gamma)$**

この時の遷移確率行列は次のような集合で定義されます，

$$
\begin{array}{r}
\mathcal{P}=\otimes \mathcal{P}_{s, a}, \text { where, } \mathcal{P}_{s, a}=\left\{P_{s, a} \in[0,1]^{|S|}:\right. \\
\left.D\left(P_{s, a}, P_{s, a}^o\right) \leq c_r, \sum_{s^{\prime} \in S} P_{s, a}\left(s^{\prime}\right)=1\right\}
\end{array}
$$

ここで$D$は任意の確率間の距離を測れる関数(ex.KL,TV)で,$c_r$は閾値,$P_{s, a}^o$を真の遷移確率行列とします．

例えばTotal VariationをDに適用したときの式は次のようになります．

$$
D_{\mathrm{tv}}\left(P_{s, a}, P_{s, a}^o\right)=(1 / 2)\left\|P_{s, a}-P_{s, a}^o\right\|_1 .
$$

そして,次のような記法を考えます．

$$
\sigma_B(v)=\inf \left\{u^{\top} v: u \in \mathcal{B}\right\}
$$

上の集合$\mathcal{B}$は上で考えた不確実な遷移確率行列の集合です．

つまり，不確実な遷移確率の集合から1番価値関数を低く推定してくれるPを選ぶということです．

この記法を使い，robust MDPの下でのベルマン作用素は次のように定義されます．

$$
T(V)(s)=\max _a(r(s, a)+\gamma \sigma_{\mathcal{P}_(s,a)})(V))
$$.

---

現実世界では真の遷移確率行列は未知の場合が多いので，モデルベース強化学習の文脈では次のように遷移確率行列を推定します．

$$
\widehat{P}_{s, a}^o\left(s^{\prime}\right)=N\left(s, a, s^{\prime}\right) / N(s,a)
$$

N(s,a)は(s,a)が何回訪れられたかという関数です．

上の式を使い，不確実な遷移確率行列の集合の定義式を書き直すと，

$$
\begin{gathered}
\widehat{\mathcal{P}}=\otimes \widehat{\mathcal{P}}_{s, a}, \text { where, } \widehat{\mathcal{P}}_{s, a}=\left\{P \in[0,1]^{\mathcal{S}}:\right. \\
\left.D\left(P_{s, a}, \widehat{P}_{s, a}\right) \leq c_r, \sum P_{s, a}\left(s^{\prime}\right)=1\right\}
\end{gathered}
$$



---

### **サンプル効率の導出**

今回はTotal Variationを使ったものを説明します．

まず，経験robust MDPを次のように定義します．

**$\widehat{M} = (S,A,r,\widehat{P},\gamma)$**

経験robust MDPを適用した価値関数は$\widehat{V}$のように表記します．

robust MDPでの目標は$\left\|V^*-V^{\pi_K}\right\|$,が十分小さくなるまでに必要なサンプル数を求めることです．

三角不等式を使い，上式を書き直すと，
$$
\left\|V^*-V^{\pi_K}\right\| \leq\left\|V^*-\widehat{V}^*\right\|+\left\|\widehat{V}^*-\widehat{V}^{\pi_K}\right\|+\left\|\widehat{V}^{\pi_K}-V^{\pi_K}\right\|,
$$


式の2項目は$\gamma$縮小性を使えば，$\left\|\widehat{V}^*-\widehat{V}^{\pi_K}\right\| \leq 2 \gamma^{K+1} /(1-\gamma)^2$ のように解けます．

なので，1項目と3項目を解くことにテクニックが必要です．

まず，$V-\widehat{V}$の項を書き直すと，

$\begin{aligned} & V^\pi(s)-\widehat{V}^\pi(s)=\gamma \sigma_{\mathcal{P}_{x, a}}\left(V^\pi\right)-\gamma \sigma_{\overline{\mathcal{p}}_{x, a}}\left(\widehat{V}^\pi\right) \\ & =\gamma\left(\sigma_{\mathcal{P}_{x, a}}\left(V^\pi\right)-\sigma_{\mathcal{P}_{x, a}}\left(\widehat{V}^\pi\right)\right)+\gamma\left(\sigma_{\mathcal{P}_{x, a}}\left(\widehat{V}^\pi\right)-\sigma_{\overline{\mathcal{P}}_{x, a}}\left(\widehat{V}^\pi\right)\right)\end{aligned}$

1項目の$\gamma\left(\sigma_{\mathcal{P}_{s, a}}\left(V^\pi\right)-\sigma_{\mathcal{P}_{s, a}}\left(\widehat{V}^\pi\right)\right)$は$\left|\sigma_{\mathcal{P}_{s, a}}\left(V_1\right)-\sigma_{\mathcal{P}_{s, a}}\left(V_2\right)\right| \leq\left\|V_1-V_2\right\|$なことが直ちに示せます．


このことから，2項目の$\sigma_{\mathcal{P}_{s, a}}\left(\widehat{V}^\pi\right)-\sigma_{\widehat{\mathcal{P}}_{s, a}}\left(\widehat{V}^\pi\right)$を解析することが大切になってきます．



$\mathcal{V}=\left\{V \in \mathbb{R}^{|\mathcal{S}|}\right.$ : $\|V\| \leq 1 /(1-\gamma)\}$.のような関数のクラスを考え，任意の$(s, a) \in \mathcal{S} \times \mathcal{A}$  ，$V \in \widehat{\mathcal{V}}$で次のようなことが成り立ちます．
$$
\left|\sigma_{\hat{\mathcal{P}}_{s, a}^{\text {tv }}}(V)-\sigma_{\mathcal{P}_{s, a}^{\text {tv }}}(V)\right| \leq 2 \max _{\mu \in \mathcal{V}}\left|\widehat{P}_{s, a} \mu-P_{s, a}^o \mu\right|
$$

$\max _{\mu \in \mathcal{V}}\left|\widehat{P}_{s, a} \mu-P_{s, a}^o \mu\right|$の項はHoeffidingの不等式と，Covering Numberを使い，次のように抑えられます．

$$
\begin{aligned}
\max _{\mu: 0 \leq \mu \leq V} & \max _{s, a}\left|\widehat{P}_{s, a} \mu-P_{s, a}^o \mu\right| \leq \\
& \frac{1}{1-\gamma} \sqrt{\frac{|\mathcal{S}|}{2 N} \log \left(\frac{12|\mathcal{S}||\mathcal{A}|}{(\delta \eta(1-\gamma))}\right.}+2 \eta
\end{aligned}
$$

上の上界の式を使うと，
$$
\begin{aligned}
& \max _{V \in \mathcal{V}} \max _{s, a}\left|\sigma_{\hat{\mathcal{P}}_{s, a}^{t v}}(V)-\sigma_{\mathcal{P}_{s, a}^{t v}}(V)\right| \leq C_u^{\mathrm{tv}}(N, \eta, \delta), \text { where, } \\
& C_u^{\mathrm{tv}}(N, \eta, \delta)=4 \eta+ \\
& \frac{2}{1-\gamma} \sqrt{\frac{|\mathcal{S}| \log (6|\mathcal{S}||\mathcal{A}| /(\delta \eta(1-\gamma)))}{2 N}}
\end{aligned}
$$

このことから，
$$
\left\|V^*-\widehat{V}^*\right\| \leq \frac{\gamma}{(1-\gamma)} C_u^{\mathrm{tv}}(N, \eta, \delta)
$$

と分かりましたね．

この解析ができれば，他の過程は簡単なので，省略し，結果だけ示すこtにします．

最初の目標であった式$|V^*-V^{\pi_K}|$は次のように書けます．


$$
\left\|V^*-V^{\pi_k}\right\| \leq \frac{2 \gamma^{k+1}}{(1-\gamma)^2}+\frac{4 \gamma}{(1-\gamma)^2} \sqrt{\frac{|\mathcal{S}| \log (6|\mathcal{S}||\mathcal{A}| /(\delta \eta(1-\gamma)))}{2 N}}+\frac{8 \gamma \eta}{(1-\gamma)}
$$

---

### **コードを書きます．**

In [20]:
import numpy as np
from functools import partial
from typing import Optional,NamedTuple
import jax.numpy as jnp
from jax.random import PRNGKey
import jax

class MDP(NamedTuple):
    S_array : jnp.ndarray
    A_array : jnp.ndarray
    P: jnp.ndarray
    rew: jnp.ndarray
    gamma : float
    H : int
    optimal_Q : Optional[jnp.ndarray] = None
    robust_optimal_Q : Optional[jnp.ndarray] = None

    @property
    def S(self):
        return len(self.S_array)
    
    @property
    def A(self):
        return  len(self.A_array)
    

S = 5
A = 3

#報酬と遷移確率の行列を作る．
key = PRNGKey(0)
key,_ = jax.random.split(key)
rew = jax.random.uniform(key,shape=(S,A))


P = np.random.rand(S*A, S) #遷移確率
P = P / np.sum(P,axis=-1,keepdims=True)
P = P.reshape(S,A,S)
np.testing.assert_allclose(P.sum(axis=-1),1)
# P = jnp.array(P)



S_array = jnp.arange(S)
A_array = jnp.arange(A)
gamma = 0.99
horizon = 1 / (1-gamma)
horizon = int(horizon)

mdp = MDP(S_array,A_array,P,rew,gamma,horizon)


In [21]:
from functools import partial
import chex

#最適価値関数の計算

@partial(jax.jit,static_argnames=('S','A'))
def _compute_optimal_Q(mdp:MDP,S:int,A:int):
    Q = jnp.zeros((S,A))
    def backup(Q):
        V = Q.max(axis=-1)
        return mdp.rew + mdp.gamma * mdp.P @ V
    body_fn = lambda i,Q:backup(Q)
    Q = jax.lax.fori_loop(0,mdp.H+100,body_fn,Q)
    return Q

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp,mdp.S,mdp.A)
optimal_Q = compute_optimal_Q(mdp)
mdp = mdp._replace(optimal_Q=optimal_Q)

In [22]:
#経験遷移確率行列
def Calc_Emp_P(mdp:MDP,N:int):
    S,A = mdp.S,mdp.A
    emp_P = jnp.zeros((S,A,S))
    N_s_a = jnp.zeros((S,A))
    N_s_a_s = jnp.zeros((S,A,S))

    for s in range(S):
        for a in range(A):
            for _ in range(N):
                s_next = np.random.choice(mdp.S,p=mdp.P[s,a])
                N_s_a = N_s_a.at[s,a].add(1)
                N_s_a_s = N_s_a_s.at[s,a,s_next].add(1)
            emp_P = emp_P.at[s,a,:].set(N_s_a_s[s,a,s_next] / N_s_a[s,a])



In [61]:
def TV(p:jnp.ndarray,p_dash:jnp.ndarray):
    return (0.5 * jnp.sum(jnp.abs(p - p_dash))).item()

In [62]:
import pulp
def Calc_Uncertainty_P(mdp:MDP,emp_P:jnp.ndarray,V:jnp.ndarray,c:float):
    S,A = mdp.S,mdp.A
    problem = pulp.LpProblem('Uncertainty_P',pulp.LpMinimize)
    P_keys = [(s,a,s_next) for s in range(S) for a in range(A) for s_next in range(S)]
    P = pulp.LpVariable.dicts('P_keys',P_keys,0,1,cat='Continuous')
    problem += pulp.lpSum([pulp.lpSum([pulp.lpSum([P[s,a,s_next] * V[s_next] for s_next in range(S)]) for a in range(A)]) for s in range(S)])

    # problem += pulp.lpSum([P[:,:,s_next] for s_next in range(S)]) == 1
    # problem += [pulp.lpSum(P[(:,:,s_next)]) for s_next in range(S)] == 1
    for s_next in range(S):
        problem += pulp.lpSum([P[s,a,s_next] for s in range(S) for a in range(A)]) == 1
    # problem += pulp.lpSum(TV(P[s,a,:],emp_P[s,a,:]) <= c for s in range(S) for a in range(A))
    for s in range(S):
        for a in range(A):
            problem += pulp.lpSum(TV(P[s,a,s_next],emp_P[s,a,s_next]) for s_next in range(S)) <= c
    problem.solve()
    P = jnp.array([[P[s,a,s_next].value() for s_next in range(S)] for a in range(A) for s in range(S)])
    P = P.reshape(S,A,S)
    return P



In [63]:
def REVI(mdp:MDP,N:int,c:float,K:int):
    S,A = mdp.S,mdp.A

    emp_P = Calc_Emp_P(mdp,N)
    Q = jnp.zeros((S,A))
    for _ in range(K):

        V = compute_optimal_Q(mdp).max(axis=-1)
        P = Calc_Uncertainty_P(mdp,emp_P,V,c)
        mdp = mdp._replace(P=P)
        Q = mdp.rew + mdp.gamma * mdp.P @ V

    return Q

Q = REVI(mdp,100,0.1,100)

TypeError: 'NoneType' object is not subscriptable